In [1]:
import pandas as pd
import numpy as np
import swifter
import gc
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 1500)
pd.set_option('display.width', 1000)
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
train = pd.read_pickle('temp_train_ml2')

In [ ]:
train['slash_deep'] = 0
train['extdeep'] = 0

train['slash_deep'] = train['TEXT_after_slash_full'].swifter.apply(lambda x: x.count(' ')+1)
train['extdeep'] = train['TEXT_extdomain'].swifter.apply(lambda x: len(x.split(' '))-1)

train['target_slash_deep'] = 0
train['target_extdeep'] = 0

train['target_slash_deep'] = train['TEXT_target_after_slash_full'].swifter.apply(lambda x: x.count(' ')+1)
train['target_extdeep'] = train['TEXT_target_extdomain'].swifter.apply(lambda x: len(x.split(' '))-1)

In [ ]:
train.loc[train['extdeep']>0,'base_domain'] =  train.loc[train['extdeep']>0,'TEXT_extdomain'].swifter.apply(lambda x: x.split(' ')[-2])
train.loc[train['extdeep']>1,'l2_domain'] = train.loc[train['extdeep']>1,'TEXT_extdomain'].swifter.apply(lambda x: x.split(' ')[-3])
train.loc[train['extdeep']>2,'l3_domain'] = train.loc[train['extdeep']>2,'TEXT_extdomain'].swifter.apply(lambda x: x.split(' ')[-4])
train.loc[train['extdeep']>3,'l4_domain'] = train.loc[train['extdeep']>3,'TEXT_extdomain'].swifter.apply(lambda x: x.split(' ')[-5])

train['zone'] = train['TEXT_extdomain'].swifter.apply(lambda x: x.split(' ')[-1])

train.loc[train['target_extdeep']>0,'target_base_domain'] =  train.loc[train['target_extdeep']>0,'TEXT_target_extdomain'].swifter.apply(lambda x: x.split(' ')[-2])
train.loc[train['target_extdeep']>1,'target_l2_domain'] = train.loc[train['target_extdeep']>1,'TEXT_target_extdomain'].swifter.apply(lambda x: x.split(' ')[-3])
train.loc[train['target_extdeep']>2,'target_l3_domain'] = train.loc[train['target_extdeep']>2,'TEXT_target_extdomain'].swifter.apply(lambda x: x.split(' ')[-4])
train.loc[train['target_extdeep']>3,'target_l4_domain'] = train.loc[train['target_extdeep']>3,'TEXT_target_extdomain'].swifter.apply(lambda x: x.split(' ')[-5])

train['target_zone'] = train['TEXT_target_extdomain'].swifter.apply(lambda x: x.split(' ')[-1])

for i in range(1,9):
    train.loc[train['slash_deep']>=i,'slash{}'.format(i)] = train.loc[train['slash_deep']>=i,'TEXT_after_slash_full'].swifter.apply(lambda x: x.split(' ')[i-1])
for i in range(1,9):
    train.loc[train['target_slash_deep']>=i,'target_slash{}'.format(i)] = train.loc[train['target_slash_deep']>=i,'TEXT_target_after_slash_full'].swifter.apply(lambda x: x.split(' ')[i-1])

In [ ]:
print(list(train.columns))

In [ ]:
['base_domain', 'l2_domain', 'l3_domain', 'l4_domain', 'zone', 'target_base_domain', 'target_l2_domain', 'target_l3_domain', 'target_l4_domain',
'target_zone', 'slash1', 'slash2', 'slash3', 'slash4', 'slash5', 'slash6', 'slash7', 'slash8', 'target_slash1', 'target_slash2', 'target_slash3',
'target_slash4', 'target_slash5', 'target_slash6', 'target_slash7', 'target_slash8']

In [ ]:
for column in ['base_domain', 'l2_domain', 'l3_domain', 'l4_domain', 'zone', 'target_base_domain', 'target_l2_domain', 'target_l3_domain', 'target_l4_domain',
'target_zone', 'slash1', 'slash2', 'slash3', 'slash4', 'slash5', 'slash6', 'slash7', 'slash8', 'target_slash1', 'target_slash2', 'target_slash3',
'target_slash4', 'target_slash5', 'target_slash6', 'target_slash7', 'target_slash8']:
    print(column)
    train[column] = train[column].factorize()[0]+2
    train[column] = train[column].factorize()[0]+1
    c_min = train[column].min()
    c_max = train[column].max()
    if str(train[column].dtype)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            train[column] = train[column].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            train[column] = train[column].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            train[column] = train[column].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
            train[column] = train[column].astype(np.int64)  
    else:
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
            train[column] = train[column].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            train[column] = train[column].astype(np.float32)
        else:
            train[column] = train[column].astype(np.float64)

In [ ]:
del train['target_1extdomain']

In [ ]:
del train['slash_1']
del train['slash_2']

In [ ]:
cats = ['extdomain',
 'after_slash_full',
 'ext2',
 'target_extdomain',
 'target_after_slash_full',
 'target_ext1',
 'inter_text_after_slash_TEXT',
 'category_1',
 'category_2',
 'target_category_1',
 'target_category_2'] + ['base_domain', 'l2_domain', 'l3_domain', 'l4_domain', 'zone', 'target_base_domain', 'target_l2_domain', 'target_l3_domain', 'target_l4_domain',
'target_zone', 'slash1', 'slash2', 'slash3', 'slash4', 'slash5', 'slash6', 'slash7', 'slash8', 'target_slash1', 'target_slash2', 'target_slash3',
'target_slash4', 'target_slash5', 'target_slash6', 'target_slash7', 'target_slash8']

In [ ]:
for col in ['TEXT_inter_text_2slash_TEXT',
 'TEXT_extdomain',
 'TEXT_after_slash_full',
 'TEXT_ext1',
 'TEXT_ext',
 'TEXT_target_extdomain',
 'TEXT_target_after_slash_full',
 'TEXT_target_ext']:
    del train[col]

In [ ]:
for col in cats:
    print(col)
    
    if 'target' in col:
        cat_packer = train[train['is_referrer']!=2][col].value_counts().reset_index()
        na_cat = int(train[col].max())
        z = list(set(cat_packer.loc[cat_packer[col]>=45,'index'].values).intersection(set(train[train['is_referrer']==2][col].unique())))
        train.loc[~train[col].isin(z),col] = na_cat
    else:
        cat_packer = train[train['is_referrer']!=2][col].value_counts().reset_index()
        na_cat = int(train[col].max())
        z = list(set(cat_packer.loc[cat_packer[col]>=30,'index'].values).intersection(set(train[train['is_referrer']==2][col].unique())))
        train.loc[~train[col].isin(z),col] = na_cat

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        if col not in ['url', 'referrer_num']:
            col_type = df[col].dtype

            if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)  
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
            else:
                pass

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
reduce_mem_usage(train)

In [ ]:
share = 0.01
dfx = train.sample(int(train.shape[0]*share))
dfx = dfx.T.drop_duplicates().T
train = train[dfx.columns]

In [ ]:
train

In [ ]:
train.to_pickle('temp_train_ml3')

In [6]:
train = pd.read_pickle('temp_train_ml3')

In [7]:
cats = ['extdomain',
 'after_slash_full',
 'ext2',
 'target_extdomain',
 'target_after_slash_full',
 'target_ext1',
 'inter_text_after_slash_TEXT',
 'category_1',
 'category_2',
 'target_category_1',
 'target_category_2',
 'base_domain',
 'l2_domain',
 'l3_domain',
 'zone',
 'target_base_domain',
 'target_l2_domain',
 'target_l3_domain',
 'target_zone',
 'slash1',
 'slash2',
 'slash3',
 'target_slash1',
 'target_slash2',
 'target_slash3',
 'target_slash4',
 'past1_base_domain',
 'past2_base_domain',
 'future1_base_domain',
 'past1_zone',
 'future-1_zone',
 'past2_zone',
 'past1_after_slash_full',
 'future-1_after_slash_full',
 'past2_after_slash_full',
 'past1_after_slash_2',
 'future-1_after_slash_2',
 'future-1_after_slash_3',
 'past2_after_slash_3',
 'past1_after_slash_4',
 'future-1_after_slash_4',
 'past2_after_slash_4',
 'past1_after_slash_5',
 'future-1_after_slash_5',
 'past2_after_slash_5',
 'event_past1_target_domain',
 'event_past2_target_domain',
 'event_future1_target_domain']

In [8]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import KFold

drops = []

cat_features = [train[:1].drop(['event_group_id','is_referrer','device_id','timestamp','target_timestamp'] + drops,
                           axis=1,errors='ignore').columns.get_loc(col) for col in cats]

fi_cols = train[:1].drop(['event_group_id',
                'is_referrer',
                'referrer_num','device_id','timestamp','target_timestamp'] + drops,axis=1,errors='ignore').columns

devices = list(train[train['is_referrer']!=2][['count_extdomain','referrer_num','is_referrer','count_device_id','timestamp','device_id']].sort_values(['count_extdomain','referrer_num','is_referrer','count_device_id','timestamp'])['device_id'].drop_duplicates().values)

In [9]:
#tpred = pd.DataFrame()
end = 10
for i in range(end):
    print(i)
    val_devices = devices[i::end].copy()
    print(len(val_devices))
    train_devices = list(set(devices) - set(val_devices))
    print(len(train_devices))
    gc.collect()

    train_data = Pool(data=train[(train['device_id'].isin(train_devices))&(train['is_referrer']!=2)].drop(['is_referrer','device_id','event_group_id','timestamp','target_timestamp']+ drops,
                                                                             axis=1,errors='ignore'),
                      label=train.loc[(train['device_id'].isin(train_devices))&(train['is_referrer']!=2),'is_referrer'].astype(int), cat_features=cats)

    valid_data = Pool(data=train[train['device_id'].isin(val_devices)].drop(['is_referrer','device_id','event_group_id','timestamp','target_timestamp'] + drops,
                                                                              axis=1,errors='ignore'),
                      label=train.loc[train['device_id'].isin(val_devices),'is_referrer'].astype(int),cat_features = cats)
    gc.collect()
    
    params = {'max_depth':16,
          'loss_function':'Logloss',
          'eval_metric':'AUC',
          'verbose': 50,
          'random_seed': 77,
          'learning_rate':0.04,
          'iterations':1500,
          'task_type':'GPU',
          'min_data_in_leaf':128,
          'one_hot_max_size' : 0}
    model = CatBoostClassifier(**params)
    model.fit(train_data,eval_set=valid_data, use_best_model=True, early_stopping_rounds=100)
    
    if i == 0:
        test = train[train['is_referrer']==2][['event_group_id','referrer_num']].copy()
        #tpred = pd.DataFrame()
        #tpred.index = train[(train['is_referrer']!=2)].index
        
    test_devices = list(train.loc[train['is_referrer']==2,'device_id'].values)
    test['pred_{}'.format(i)] = model.predict_proba(Pool(data=train[train['device_id'].isin(test_devices)].drop(['is_referrer','device_id','event_group_id','timestamp','target_timestamp'] + drops,
                                                                              axis=1,errors='ignore'),cat_features = cats))[:, 1]
    #tpred['pred_{}'.format(i)] = model.predict_proba(Pool(data=train[(train['is_referrer']!=2)].drop(['is_referrer','device_id','referrer_num','event_group_id','timestamp','target_timestamp']+ drops,
    #                                                                         axis=1,errors='ignore'), cat_features=cats))[:, 1]
    
    test.to_pickle('new_test_{}'.format(i))
    #tpred.to_pickle('new_tpred_{}'.format(i))
    gc.collect()

0
20000
180000
0:	test: 0.9054546	best: 0.9054546 (0)	total: 11.3s	remaining: 4h 43m 13s
50:	test: 0.9650715	best: 0.9650715 (50)	total: 13m 13s	remaining: 6h 15m 33s
100:	test: 0.9707483	best: 0.9707483 (100)	total: 28m 17s	remaining: 6h 31m 57s


KeyboardInterrupt: 

In [5]:
#tpred = pd.DataFrame()
end = 10
for i in range(end):
    print(i)
    val_devices = devices[i::end].copy()
    print(len(val_devices))
    train_devices = list(set(devices) - set(val_devices))
    print(len(train_devices))
    gc.collect()

    train_data = Pool(data=train[(train['device_id'].isin(train_devices))&(train['is_referrer']!=2)].drop(['is_referrer','device_id','referrer_num','event_group_id','timestamp','target_timestamp']+ drops,
                                                                             axis=1,errors='ignore'),
                      label=train.loc[(train['device_id'].isin(train_devices))&(train['is_referrer']!=2),'is_referrer'].astype(int), cat_features=cats)

    valid_data = Pool(data=train[train['device_id'].isin(val_devices)].drop(['is_referrer','device_id','referrer_num','event_group_id','timestamp','target_timestamp'] + drops,
                                                                              axis=1,errors='ignore'),
                      label=train.loc[train['device_id'].isin(val_devices),'is_referrer'].astype(int),cat_features = cats)
    gc.collect()
    
    params = {'max_depth':16,
          'loss_function':'Logloss',
          'eval_metric':'AUC',
          'verbose': 50,
          'random_seed': 77,
          'learning_rate':0.04,
          'iterations':1500,
          'task_type':'GPU',
          'min_data_in_leaf':128,
          'one_hot_max_size' : 0}
    model = CatBoostClassifier(**params)
    model.fit(train_data,eval_set=valid_data, use_best_model=True, early_stopping_rounds=100)
    
    if i == 0:
        test = train[train['is_referrer']==2][['event_group_id','referrer_num']].copy()
        #tpred = pd.DataFrame()
        #tpred.index = train[(train['is_referrer']!=2)].index
        
    test_devices = list(train.loc[train['is_referrer']==2,'device_id'].values)
    test['pred_{}'.format(i)] = model.predict_proba(Pool(data=train[train['device_id'].isin(test_devices)].drop(['is_referrer','device_id','referrer_num','event_group_id','timestamp','target_timestamp'] + drops,
                                                                              axis=1,errors='ignore'),cat_features = cats))[:, 1]
    #tpred['pred_{}'.format(i)] = model.predict_proba(Pool(data=train[(train['is_referrer']!=2)].drop(['is_referrer','device_id','referrer_num','event_group_id','timestamp','target_timestamp']+ drops,
    #                                                                         axis=1,errors='ignore'), cat_features=cats))[:, 1]
    
    test.to_pickle('new_test_{}'.format(i))
    #tpred.to_pickle('new_tpred_{}'.format(i))
    gc.collect()

0
20000
180000
0:	test: 0.9058609	best: 0.9058609 (0)	total: 11.5s	remaining: 4h 46m 58s
50:	test: 0.9649886	best: 0.9649886 (50)	total: 13m 23s	remaining: 6h 20m 19s
100:	test: 0.9709545	best: 0.9709545 (100)	total: 28m 22s	remaining: 6h 32m 58s
150:	test: 0.9736317	best: 0.9736317 (150)	total: 41m 43s	remaining: 6h 12m 48s
200:	test: 0.9748549	best: 0.9748642 (197)	total: 54m 24s	remaining: 5h 51m 36s
250:	test: 0.9758521	best: 0.9758521 (250)	total: 1h 8m 8s	remaining: 5h 39m 5s
300:	test: 0.9766777	best: 0.9766777 (300)	total: 1h 22m 12s	remaining: 5h 27m 28s
350:	test: 0.9772597	best: 0.9772597 (350)	total: 1h 35m 59s	remaining: 5h 14m 12s
400:	test: 0.9776710	best: 0.9776772 (398)	total: 1h 49m 32s	remaining: 5h 12s
450:	test: 0.9780182	best: 0.9780182 (450)	total: 2h 3m 30s	remaining: 4h 47m 16s
500:	test: 0.9782796	best: 0.9782796 (500)	total: 2h 17m 19s	remaining: 4h 33m 48s
550:	test: 0.9785504	best: 0.9785504 (550)	total: 2h 31m 51s	remaining: 4h 21m 32s
600:	test: 0.9788309

KeyboardInterrupt: 

In [ ]:
#0

In [ ]:
test

In [ ]:

fi = pd.DataFrame({'feature_importance': model.get_feature_importance(train_data),
              'feature_names': fi_cols}).sort_values(by=['feature_importance'], ascending=False)

In [ ]:
fi

In [ ]:
fi[fi['feature_importance']==0].shape

In [ ]:
train = train.drop(['past2_after_slash_2','past1_after_slash_3','slash4','target_l4_domain','l4_domain'],axis=1)

In [ ]:
test = train[train['is_referrer']==2][['event_group_id','referrer_num']].copy()
test_devices = list(train.loc[train['is_referrer']==2,'device_id'].values)

test['pred'] = model.predict_proba(Pool(data=train[train['device_id'].isin(test_devices)].drop(['is_referrer','device_id','referrer_num','event_group_id','timestamp','target_timestamp'],
                                                                          axis=1,errors='ignore'),cat_features = cats,text_features=text))[:, 1]



In [ ]:
test

In [ ]:
test.sort_values('pred',ascending=False,inplace=True)
test = test.drop_duplicates(['event_group_id'],keep='first')

In [ ]:
id_map = pd.read_pickle('id_map')
to_original_event_group_id = dict(zip(id_map['eve'],id_map['event_group_id']))
to_eve = dict(zip(id_map['event_group_id'],id_map['eve']))

test['event_group_id'] = test['event_group_id'].map(to_original_event_group_id)
test = test.sort_values('event_group_id')
test[['event_group_id','referrer_num']].to_csv('cb_exp_07.06.2022_2.csv',index=False)

In [ ]:
test

In [ ]:
model = CatBoostClassifier(**params)
model.fit(valid_data,eval_set=train_data, use_best_model=True, early_stopping_rounds=200)

In [ ]:
fi2 = pd.DataFrame({'feature_importance': model.get_feature_importance(valid_data),
              'feature_names': fi_cols}).sort_values(by=['feature_importance'], ascending=False)

In [ ]:
fi2

In [ ]:
train = train.drop(list(fi.loc[fi['feature_importance']<0.001,'feature_names'].values),axis=1)

In [ ]:
train.info()

In [ ]:
%%time
train = pd.read_csv('semrush_cup_train_data.csv')
val = pd.read_csv('semrush_cup_test_data.csv')
train['is_referrer'] = train['is_referrer'].astype(int)
val['is_referrer'] = 2
train = pd.concat([train,val]).reset_index(drop=True)
train.loc[train['domain'].isna(),'domain'] = train.loc[train['domain'].isna(),'url'].str.split('/').str[0]

train = train.sort_values(['timestamp']).reset_index(drop=True)
eve = train.sort_values(['timestamp'])[['event_group_id']].drop_duplicates(keep='first').reset_index(drop=True).reset_index()
eve.columns = ['eve','event_group_id']
dev = train.sort_values(['timestamp'])[['device_id']].drop_duplicates(keep='first').reset_index(drop=True).reset_index()
dev.columns = ['dev','device_id']
train = train.merge(eve,on='event_group_id',how='left')
train = train.merge(dev,on='device_id',how='left')

train = train.sort_values(['event_group_id','referrer_num']).reset_index(drop=True)
train['event_group_id'] = train['eve']
train['device_id'] = train['dev']
del train['eve'], train['dev'], eve, dev
train = train.sort_values('timestamp').reset_index(drop=True)
gc.collect()

In [ ]:
train['extdomain'] = train['url'].swifter.apply(lambda x: x.split('/')[0])
train['after_slash_full'] = train['url'].swifter.apply(lambda x: '/'.join(x.split('/')[1:]))
train['ext1'] = train.swifter.apply(lambda row : row['extdomain'].replace(str(row['domain']), ''), axis=1)
train['1extdomain'] =  train['extdomain'].swifter.apply(lambda x: x.split('.')[-2] + '.' + x.split('.')[-1])
train['ext'] = train.swifter.apply(lambda row : row['extdomain'].replace(str(row['1extdomain']), ''), axis=1)
train['ext2'] = train.swifter.apply(lambda row : row['ext'].replace(str(row['ext1']), ''), axis=1)
train['slash_1'] = train['after_slash_full'].swifter.apply(lambda x: x.split('/')[0])
train['1extdomain'] =  train['extdomain'].swifter.apply(lambda x: x.split('.')[-2] + '.' + x.split('.')[-1])
train['zone'] = train['extdomain'].swifter.apply(lambda x: x.split('.')[-1])
#zone share
#'1extdomain' share

In [ ]:
%%time
for col in ['extdomain','1extdomain','after_slash_full','ext1','ext2','slash_1','zone']:
    print(col)
    train = train.merge(train[train['referrer_num']=='target'][['event_group_id',col]].rename(columns={col:'target_temp'}),
                        on = 'event_group_id', how = 'left')
    print('0')
    for p in range(1,4):
        temp = train[train['referrer_num']=='target'].sort_values(['timestamp'])[['event_group_id','device_id',col]]
        temp['eve_past_tar_{}_{}'.format(col,p)] = temp.groupby(['device_id'])[col].shift(p).values
        temp.columns = ['event_group_id','device_id',col,'eve_past_tar_{}_{}'.format(col,p)]
        train = train.merge(temp[['event_group_id','eve_past_tar_{}_{}'.format(col,p)]],
                            on='event_group_id',
                            how='left')
        train['eve_past_tar_even_how_current_{}_{}'.format(col,p)] = 0
        train.loc[train['target_temp'] == train['eve_past_tar_{}_{}'.format(col,p)],'eve_past_tar_even_how_current_{}_{}'.format(col,p)] = 1
    print('future')      
    for t in range(-4,0):
        temp = train[train['referrer_num']=='target'].sort_values(['timestamp'])[['event_group_id','device_id',col]]
        temp['eve_future_tar_{}_{}'.format(col,t)] = temp.groupby(['device_id'])[col].shift(t).values
        temp.columns = ['event_group_id','device_id',col,'eve_future_tar_{}_{}'.format(col,t)]
        train = train.merge(temp[['event_group_id','eve_future_tar_{}_{}'.format(col,t)]],on='event_group_id',how='left')
        train['eve_future_tar_even_how_current_{}_{}'.format(col,t)] = 0
        train.loc[train['target_temp'] == train['eve_future_tar_{}_{}'.format(col,t)],'eve_future_tar_even_how_current_{}_{}'.format(col,t)] = 1
    
    del train['target_temp']
    gc.collect()

In [ ]:
%%time
past_cols = ['extdomain','1extdomain','after_slash_full','ext1','ext2','slash_1','zone']

past_for_target = []

for col in past_cols:
    print(col)
    
    train = train.merge(train[train['referrer_num']=='target'][['event_group_id',col]].rename(columns={col:'target_temp'}),
                        on = 'event_group_id', how = 'left')

    for r in range(1,4):
        
        train['past{}_{}'.format(r,col)] = train[['event_group_id',col]].groupby(['event_group_id'])[col].shift(r)
        train['past_{}_how_current_{}'.format(r,col)] = 0
        train.loc[train['target_temp'] == train['past{}_{}'.format(r,col)],'past_{}_how_current_{}'.format(r,col)] = 1

        train['past{}_{}'.format(r,col)] = train['past{}_{}'.format(r,col)].factorize()[0]+2
        train['past{}_{}'.format(r,col)] = train['past{}_{}'.format(r,col)].factorize()[0]+1
        
        past_for_target = past_for_target + ['past_{}_how_current_{}'.format(r,col)]
        
        train['future{}_{}'.format(0-r,col)] = train[['event_group_id',col]].groupby(['event_group_id'])[col].shift(0-r)
        train['future_{}_how_current_{}'.format(0-r,col)] = 0
        train.loc[train['target_temp'] == train['future{}_{}'.format(0-r,col)],'future_{}_how_current_{}'.format(0-r,col)] = 1        
        
        past_for_target = past_for_target + ['future_{}_how_current_{}'.format(0-r,col)]
        
        """
        train['future{}_{}'.format(0-r,col)] = train['future{}_{}'.format(0-r,col)].factorize()[0]+2
        train['future{}_{}'.format(0-r,col)] = train['future{}_{}'.format(0-r,col)].factorize()[0]+1
        
        for column in [ 'past{}_{}'.format(r,col),
                        'past_{}_how_current_{}'.format(r,col),
                        'future{}_{}'.format(0-r,col),
                        'future_{}_how_current_{}'.format(0-r,col)]:
            
            c_min = train[column].min()
            c_max = train[column].max()

            if str(train[column].dtype)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train[column] = train[column].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train[column] = train[column].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train[column] = train[column].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train[column] = train[column].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train[column] = train[column].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train[column] = train[column].astype(np.float32)
                else:
                    train[column] = train[column].astype(np.float64)"""

        
    del train['target_temp']

In [ ]:
%%time
cats = pd.read_pickle('cats_full')
cats = cats[['domain',
 'category_1',
 'category_2',
 'cat_main_domain_18-24_mean',
 'cat12_cnt',
 'cat_main_domain_55-64_min',
 'cat_main_domain_M_mean',
 'cat_main_domain_55-64_mean',
 'cat12md_cnt',
 'cat_main_domain_35-44_mean',
 'cat_main_domain_65+_mean',
 'cat_main_domain_25-34_std',
 'cat_main_domain_65+_max',
 'cats_1_35-44_max',
 'cats_1_zoner4_F_mean']]

train = train.merge(cats,on='domain',how='left')

for r in range(1,4):
    print(r)
    train['deltatime_category_1{}'.format(r)] = train[['category_1','timestamp']].groupby(['category_1'])['timestamp'].diff(r).fillna(0-r)
    train['deltatime_category_2{}'.format(r)] = train[['category_2','timestamp']].groupby(['category_2'])['timestamp'].diff(r).fillna(0-r)
    train['deltatime_device_cat1{}'.format(r)] = train[['device_id','category_1','timestamp']].groupby(['device_id','category_1'])['timestamp'].diff(r).fillna(0-r)
    train['deltatime_device_cat2{}'.format(r)] = train[['device_id','category_2','timestamp']].groupby(['device_id','category_2'])['timestamp'].diff(r).fillna(0-r)  

In [ ]:
train

In [ ]:
cols = ['event_group_id','device_id','referrer_num','is_referrer'] + [c for c in train.columns if ('delta' in c) or ('current' in c) or ('cat' in c)]

In [ ]:
%%time
target_columns = [c for c in cols if c not in ['referrer_num','is_referrer','referrer_domain','referrer_url']]
target_data = train[train['referrer_num']=='target'][target_columns]
target_data.columns = ['event_group_id','device_id'] + ['target_{}'.format(c) for c in target_data.columns[2:]]
train = train[train['referrer_num']!='target'].merge(target_data,on = ['event_group_id','device_id'],how='left')

In [ ]:
for column in [c for c in train.columns if ('delta' in c) or ('current' in c) or ('cat' in c)]:
    train[column] = train[column].factorize()[0]+2
    train[column] = train[column].factorize()[0]+1
    c_min = train[column].min()
    c_max = train[column].max()
    if str(train[column].dtype)[:3] == 'int':
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            train[column] = train[column].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            train[column] = train[column].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            train[column] = train[column].astype(np.int32)
        elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
            train[column] = train[column].astype(np.int64)  
    else:
        if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
            train[column] = train[column].astype(np.float16)
        elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            train[column] = train[column].astype(np.float32)
        else:
            train[column] = train[column].astype(np.float64)

In [ ]:
train

In [ ]:
['category_1', 'category_2','target_category_1','target_category_2']


In [ ]:
for col in ['category_1', 'category_2','target_category_1','target_category_2']:
    print(col)
    
    if 'target' in col:
        cat_packer = train[train['is_referrer']!=2][col].value_counts().reset_index()
        na_cat = int(train[col].max() + 1)
        z = list(set(cat_packer.loc[cat_packer[col]>=18,'index'].values).intersection(set(train[train['is_referrer']==2][col].unique())))
        train.loc[~train[col].isin(z),col] = na_cat
    else:
        cat_packer = train[train['is_referrer']!=2][col].value_counts().reset_index()
        na_cat = int(train[col].max() + 1)
        z = list(set(cat_packer.loc[cat_packer[col]>=2,'index'].values).intersection(set(train[train['is_referrer']==2][col].unique())))
        train.loc[~train[col].isin(z),col] = na_cat

In [ ]:
train

In [ ]:
for col in [c for c in train.columns if ('delta' in c) or ('current' in c) or ('cat' in c)]:
    train2[col] = train[col].values

In [ ]:
train2